In [198]:
import pandas as pd
import numpy as np
import os
from scipy import stats

# visualize
import seaborn as sns
import matplotlib.pyplot as plt

# acquire
from env import host, user, password
from pydataset import data

# TIDY DATA

### 1. Attendance Data

- Load the attendance.csv file and calculate an attendnace percentage for each student. One half day is worth 50% of a full day, and 10 tardies is equal to one absence.


In [199]:
attendance = pd.read_csv('untidy-data/attendance.csv')
attendance

,Unnamed: 0,2018-01-01,2018-01-02,2018-01-03,2018-01-04,2018-01-05,2018-01-06,2018-01-07,2018-01-08
0,Sally,P,T,T,H,P,A,T,T
1,Jane,A,P,T,T,T,T,A,T
2,Billy,A,T,A,A,H,T,P,T
3,John,P,T,H,P,P,T,P,P


In [200]:
attendance.columns = ['name', '2018-01-01', '2018-01-02', '2018-01-03', '2018-01-04',
       '2018-01-05', '2018-01-06', '2018-01-07', '2018-01-08']

In [201]:
attendance_melt = attendance.melt(id_vars='name', var_name='date', value_name='attendance')

In [202]:
attendance_melt['grade'] = attendance_melt.attendance

In [203]:
attendance_melt.head()

,name,date,attendance,grade
0,Sally,2018-01-01,P,P
1,Jane,2018-01-01,A,A
2,Billy,2018-01-01,A,A
3,John,2018-01-01,P,P
4,Sally,2018-01-02,T,T


In [204]:
attendance_melt.grade = attendance_melt.grade.replace(['P','A','H','T'],[1, 0, .5, 0.9])
attendance_melt.head()

,name,date,attendance,grade
0,Sally,2018-01-01,P,1.0
1,Jane,2018-01-01,A,0.0
2,Billy,2018-01-01,A,0.0
3,John,2018-01-01,P,1.0
4,Sally,2018-01-02,T,0.9


In [205]:
grade = attendance_melt.pivot_table(index = 'name')
grade.reset_index(inplace = True)

In [206]:
grade

,name,grade
0,Billy,0.5250
1,Jane,0.6875
2,John,0.9125
3,Sally,0.7625


### 2. Coffee Levels

- Read the coffee_levels.csv file.
- Transform the data so that each carafe is in it's own column.
- Is this the best shape for the data?

In [209]:
coffee = pd.read_csv('untidy-data/coffee_levels.csv')
coffee.sample(10)

,hour,coffee_carafe,coffee_amount
3,11,x,0.335533
1,9,x,0.451018
25,13,z,0.398520
5,13,x,0.310711
9,17,x,0.391560
21,9,z,0.915990
6,14,x,0.507288
18,16,y,0.544676
15,13,y,0.997464
12,10,y,0.023163


In [212]:
coffee_pivot = coffee.pivot_table(index = 'hour', columns='coffee_carafe')

In [214]:
coffee_pivot

coffee_amount                    
coffee_carafe             x         y         z
hour                                           
8                  0.816164  0.189297  0.999264
9                  0.451018  0.521502  0.915990
10                 0.843279  0.023163  0.144928
11                 0.335533  0.235529  0.311495
12                 0.898291  0.017009  0.771947
13                 0.310711  0.997464  0.398520
14                 0.507288  0.058361  0.864464
15                 0.215043  0.144644  0.436364
16                 0.183891  0.544676  0.280621
17                 0.391560  0.594126  0.436677

It is not the best shape of the data for machine learning, but it is a good way to look at it

### 3. Cake Recipes

- Read the cake_recipes.csv data. This data set contains cake tastiness scores for combinations of different recipes, oven rack positions, and oven temperatures.
- Tidy the data as necessary.


In [234]:
cake_recipe = pd.read_csv('untidy-data/cake_recipes.csv')

In [235]:
cake_recipe

,recipe:position,225,250,275,300
0,a:bottom,61.738655,53.912627,74.414730,98.786784
1,a:top,51.709751,52.009735,68.576858,50.228470
2,b:bottom,57.095320,61.904369,61.196980,99.248541
3,b:top,82.455004,95.224151,98.594881,58.169349
4,c:bottom,96.470207,52.001358,92.893227,65.473084
5,c:top,71.306308,82.795477,92.098049,53.960273
6,d:bottom,52.799753,58.670419,51.747686,56.183110
7,d:top,96.873178,76.101363,59.571620,50.971626


In [236]:
cake_recipe = cake_recipe.melt(id_vars='recipe:position', var_name='temp')
cake_recipe.head()

,recipe:position,temp,value
0,a:bottom,225,61.738655
1,a:top,225,51.709751
2,b:bottom,225,57.095320
3,b:top,225,82.455004
4,c:bottom,225,96.470207


In [237]:
cake_recipe[['recipe','position']] = cake_recipe['recipe:position'].str.split(':', expand=True)

In [238]:
cake_recipe.drop(columns='recipe:position', inplace=True)

- Which recipe, on average, is the best? **recipe b**


In [239]:
cake_recipe.groupby('recipe').value.mean().nlargest(1)

recipe
b    76.736074
Name: value, dtype: float64

- Which oven temperature, on average, produces the best results? **275**

In [240]:
cake_recipe.groupby('temp').value.mean().nlargest(1)

temp
275    74.886754
Name: value, dtype: float64

- Which combination of recipe, rack position, and temperature gives the best result? **recipe b, bottom rack, 300 degre**es

In [241]:
cake_recipe.value_counts().nlargest(1, keep='first')

temp  value      recipe  position
300   99.248541  b       bottom      1
dtype: int64

# GAPMINDER1 and GAPMINDER2

In [243]:
gm1 = pd.read_csv('untidy-data/gapminder1.csv')
gm2 = pd.read_csv('untidy-data/gapminder2.csv')
list(gm2.columns)

['country',
 'life_expect_1955',
 'life_expect_1960',
 'life_expect_1965',
 'life_expect_1970',
 'life_expect_1975',
 'life_expect_1980',
 'life_expect_1985',
 'life_expect_1990',
 'life_expect_1995',
 'life_expect_2000',
 'life_expect_2005',
 'pop_1955',
 'pop_1960',
 'pop_1965',
 'pop_1970',
 'pop_1975',
 'pop_1980',
 'pop_1985',
 'pop_1990',
 'pop_1995',
 'pop_2000',
 'pop_2005']

In [244]:
gm2_life = gm2.drop(columns=['pop_1955',
 'pop_1960',
 'pop_1965',
 'pop_1970',
 'pop_1975',
 'pop_1980',
 'pop_1985',
 'pop_1990',
 'pop_1995',
 'pop_2000',
 'pop_2005'])

In [245]:
gm2_pop = gm2.drop(columns=['life_expect_1955',
 'life_expect_1960',
 'life_expect_1965',
 'life_expect_1970',
 'life_expect_1975',
 'life_expect_1980',
 'life_expect_1985',
 'life_expect_1990',
 'life_expect_1995',
 'life_expect_2000',
 'life_expect_2005'])

In [246]:
gm2_life = gm2_life.melt(id_vars='country', var_name='year', value_name='life_expect')

In [247]:
gm2_life.head()

,country,year,life_expect
0,Afghanistan,life_expect_1955,30.332
1,Argentina,life_expect_1955,64.399
2,Aruba,life_expect_1955,64.381
3,Australia,life_expect_1955,70.330
4,Austria,life_expect_1955,67.480


In [248]:
gm2_life.year = gm2_life.year.str.replace('life_expect_','')

In [249]:
gm2_life.head()

,country,year,life_expect
0,Afghanistan,1955,30.332
1,Argentina,1955,64.399
2,Aruba,1955,64.381
3,Australia,1955,70.330
4,Austria,1955,67.480


In [250]:
gm2_pop = gm2_pop.melt(id_vars='country', var_name='year', value_name='population')

In [251]:
gm2_pop.head() 

,country,year,population
0,Afghanistan,pop_1955,8891209
1,Argentina,pop_1955,18927821
2,Aruba,pop_1955,53865
3,Australia,pop_1955,9277087
4,Austria,pop_1955,6946885


In [252]:
gm2_pop.year = gm2_pop.year.str.replace('pop_','')

In [253]:
gm2_pop.head()

,country,year,population
0,Afghanistan,1955,8891209
1,Argentina,1955,18927821
2,Aruba,1955,53865
3,Australia,1955,9277087
4,Austria,1955,6946885


In [254]:
gm2_melt = gm2_life.merge(gm2_pop, how='inner', on=['country','year'])

In [182]:
gm2_melt

,country,year,life_expect,population
0,Afghanistan,1955,30.332,8891209
1,Argentina,1955,64.399,18927821
2,Aruba,1955,64.381,53865
3,Australia,1955,70.330,9277087
4,Austria,1955,67.480,6946885
...,...,...,...,...
688,Switzerland,2005,81.701,7489370
689,Turkey,2005,71.777,69660559
690,United Kingdom,2005,79.425,60441457
691,United States,2005,78.242,295734134


In [183]:
gm1.measure.unique()

array(['pop', 'life_expect', 'fertility'], dtype=object)

In [184]:
gm1.head()

,year,country,measure,measurement
0,1955,Afghanistan,pop,8891209.0
1,1960,Afghanistan,pop,9829450.0
2,1965,Afghanistan,pop,10997885.0
3,1970,Afghanistan,pop,12430623.0
4,1975,Afghanistan,pop,14132019.0


In [185]:
gm1_tidy = gm1.pivot_table(index=['country','year'], columns='measure')

In [186]:
gm1_tidy.reset_index(inplace=True)

In [255]:
gm1_tidy.sample(10)

country  year measurement                         
measure                         fertility life_expect          pop
665      United Kingdom  1980      1.8000      74.040   56314000.0
237         El Salvador  1985      3.9010      63.154    4664361.0
425             Jamaica  1990      2.8400      71.766    2347922.0
421             Jamaica  1970      5.0000      69.000    1943787.0
164            Colombia  2005      2.2205      72.889   42954279.0
342               India  1960      5.8216      43.605  434000000.0
587              Rwanda  1975      8.4920      45.000    4356863.0
253              France  1955      2.7120      68.930   43427669.0
584              Rwanda  1960      8.1000      43.000    3031804.0
191                Cuba  1975      2.1500      72.649    9290074.0